In [19]:
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
import sys
import roman
#import attributes

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.types import VARCHAR

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

import timeperiod2daterange
#import attributes

from bson.objectid import ObjectId

In [20]:
myclient = pymongo.MongoClient(config.MONGO_URI)
stagingDb = myclient[config.DB_STAGING]
analyseDb = myclient[config.DB_ANALYSE]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']


In [27]:
#grp_aggr = [{"$match" : {'soort': 'Artefact', '_id': ObjectId('62aa1843288d4dfe4c78f511')}}]
grp_aggr = [{"$match" : {'soort': 'Artefact', 'vondstID': {'$exists': True}}}]
df_artf = pd.DataFrame(list(analyseCol.aggregate(grp_aggr)))
#df_artf.loc[df_artf['brondata'].notnull() & df_artf.brondata.str.contains('VONDST', case=False)]
#df_artf['brondata']
df_artf

""


In [7]:
df = pd.concat([df_vondst[['datering_origineel', 'nieuw']].copy(), df_spoor[['datering_origineel', 'nieuw']].copy(), df_artf[['datering_origineel', 'nieuw']].copy()])
df['nieuw'] = df.apply(lambda x: str(fixDatering(x['datering_origineel'])), axis=1)

grp_df = df.groupby(['datering_origineel', 'nieuw']).agg({'datering_origineel': ['count']}).reset_index()
grp_df


Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waarde naar getal: <> met melding: string index out of range
Fout bij omzetten romeinse waard

,datering_origineel,nieuw,datering_origineel
,,,count
0,1,"(100, 100)",12
1,2,"(200, 200)",12
2,3,"(300, 300)",12
3,4,"(400, 400)",12
4,5,"(500, 500)",12
...,...,...,...
497,voor 1525,"(1525, 1525)",3
498,voor 1600,"(1600, 1600)",3
499,waarschijnlijk XIII-XIV,"(1400, 1400)",2


In [8]:
grp_df.to_excel('datering.xlsx')

In [9]:
fixDatering('15th century')

(1500, 1500)

In [10]:
timeperiod2daterange.detection2daterange('17e eeuw')

[1600, 1699]